In [1]:
!oc login --token=sha256~KiSeXt6HnPQoQX_af6KMcyGBpCPveQWp8iuNuce0e1o --server=https://api.rhods-weka.bj30.p1.openshiftapps.com:6443

Logged into "https://api.rhods-weka.bj30.p1.openshiftapps.com:6443" as "heyselbi" using the token provided.

You have access to 110 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "distributed-tf".


In [2]:
%%writefile mnist-tfjob.yaml
apiVersion: kubeflow.org/v1
kind: TFJob
metadata:
  generateName: tfjob-multi-
  namespace: distributed-tf
spec:
  cleanPodPolicy: Running
  tfReplicaSpecs:
    Worker:
      selector:
        matchLabels:
           app: distributed
      replicas: 2
      restartPolicy: Never
      template:
        metadata:
          labels:
            app: distributed
          annotations:
            sidecar.istio.io/inject: "false"
        spec:            
          containers:
          - name: tensorflow
            image: "quay.io/selbi/tf-mnist:ubi8-tf2.4.0"
            command: ["python3", "/home/mnist/src/main.py"] 
            imagePullPolicy: Always
            env:
              - name: LD_LIBRARY_PATH
                value: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/home/nccl/lib:/usr/local/cuda/lib:/usr/local/cuda-10.1/compat
              - name: TF_FORCE_GPU_ALLOW_GROWTH
                value: "true"
          imagePullSecrets:
            - name: "selbi-build-pull-secret"
          nodeSelector:
            node.kubernetes.io/instance-type: "g4dn.12xlarge"
          affinity:
            podAntiAffinity:
               requiredDuringSchedulingIgnoredDuringExecution:
               - labelSelector:
                   matchExpressions:
                   - key: app
                     operator: In
                     values:
                     - distributed
                 topologyKey: "kubernetes.io/hostname"

Writing mnist-tfjob.yaml


In [5]:
!oc create -f mnist-tfjob.yaml

tfjob.kubeflow.org/tfjob-multi-4cq8w created


In [6]:
!oc get pods | grep tfjob-multi

tfjob-multi-4cq8w-worker-0     1/1     Running   0          4s
tfjob-multi-4cq8w-worker-1     1/1     Running   0          4s


In [7]:
!oc logs pod/tfjob-multi-4cq8w-worker-0 | grep "GrpcChannelCache"

2021-07-16 14:50:24.329209: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> tfjob-multi-4cq8w-worker-0.distributed-tf.svc:2222, 1 -> tfjob-multi-4cq8w-worker-1.distributed-tf.svc:2222}


In [12]:
!oc logs pod/tfjob-multi-4cq8w-worker-0 | grep "/job:worker/replica:0/task:0/device:GPU:"

2021-07-16 14:50:24.283930: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1406] Created TensorFlow device (/job:worker/replica:0/task:0/device:GPU:0 with 13968 MB memory) -> physical GPU (device: 0, name: Tesla T4, pci bus id: 0000:00:1b.0, compute capability: 7.5)
2021-07-16 14:50:24.284996: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1406] Created TensorFlow device (/job:worker/replica:0/task:0/device:GPU:1 with 13968 MB memory) -> physical GPU (device: 1, name: Tesla T4, pci bus id: 0000:00:1c.0, compute capability: 7.5)
2021-07-16 14:50:24.286042: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1406] Created TensorFlow device (/job:worker/replica:0/task:0/device:GPU:2 with 13968 MB memory) -> physical GPU (device: 2, name: Tesla T4, pci bus id: 0000:00:1d.0, compute capability: 7.5)
2021-07-16 14:50:24.287090: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1406] Created TensorFlow device (/job:worker/replica:0/task:0/device:GPU:3 with 13968 MB memory) -> physical

In [11]:
!oc exec -it nvidia-driver-daemonset-268p9 nvidia-smi -n gpu-operator-resources

Fri Jul 16 14:50:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1B.0 Off |                    0 |
| N/A   35C    P0    28W /  70W |   1002MiB / 15109MiB |     19%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            On   | 00000000:00:1C.0 Off |                    0 |
| N/A   

In [10]:
!oc exec -it nvidia-driver-daemonset-vmxck nvidia-smi -n gpu-operator-resources

Fri Jul 16 14:50:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1B.0 Off |                    0 |
| N/A   41C    P0    29W /  70W |    992MiB / 15109MiB |     19%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            On   | 00000000:00:1C.0 Off |                    0 |
| N/A   